In [55]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
import openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain
from langchain.prompts.pipeline import PipelinePromptTemplate


from dotenv import load_dotenv

In [47]:
# load_dotenv()
OPENAI_API_KEY = os.getenv()
openai.api_key = OPENAI_API_KEY

In [35]:
examples = [
    {
        'text' : '아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요.',
        'tag': '긍정 리뷰입니다.'
    },
    {
        'text' : '선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전화했더니 바로주신다했지만 배송도 누락되어있었네요.. 확인안하고 바로 선물했으면 큰일날뻔했네요..이렇게 배송이 오래걸렸으면 사는거 다시 생각했을거같아요 아쉽네요..',
        'tag': '부정 리뷰입니다.'
    },
    {
        'text' : '처음 쓸때만 좋고 쓰다보니 의자뺄때 소리 계속 나요 그래서 붙이는부직포 덧붙였는데 여전히 끌리네요 비추에요 테니스공으로 살껄 이쁜거 산다고 이거 사서 후회되요',
        'tag': '부정 리뷰입니다.'
    },
]

In [61]:
full_template = """
{intro}
{example}
{input_prompt}
 """
full_prompt = PromptTemplate.from_template(full_template)

In [53]:
intro_template = """
너는 cs 담당자야 \n
주어진 text를 분석해서 긍정 리뷰인지 부정 리뷰인지 확인하려고 해 \n
예시 데이터는 <text> 리뷰 </text> <tag> 긍정/부정 리뷰입니다 </tag> 형식의 xml로 주어질거야 \n
예시 데이터를 보고 주어지는 리뷰가 긍정이면 '긍정 리뷰입니다', 부정이면 '부정 리뷰압니다' 로 분류해줘
"""
intro_prompt = PromptTemplate.from_template(intro_template)

In [58]:
example_template = """ 
<text>'아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요.'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전화했더니 바로주신다했지만 배송도 누락되어있었네요.. 확인안하고 바로 선물했으면 큰일날뻔했네요..이렇게 배송이 오래걸렸으면 사는거 다시 생각했을거같아요 아쉽네요..'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'처음 쓸때만 좋고 쓰다보니 의자뺄때 소리 계속 나요 그래서 붙이는부직포 덧붙였는데 여전히 끌리네요 비추에요 테니스공으로 살껄 이쁜거 산다고 이거 사서 후회되요'</text>
<tag>'부정 리뷰입니다'</tag>
"""
example_prompt = PromptTemplate.from_template(example_template)

In [54]:
input_template = """ 
<text> {input} </text>
"""
input_prompt = PromptTemplate.from_template(input_template)

In [59]:
prompts = [
    ("intro", intro_prompt),
    ("example", example_prompt),
    ("input_prompt", input_prompt),
]

In [62]:
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=prompts
)

In [72]:
with open('../data/쇼핑/naver_shopping.txt', 'r') as f:
    data = f.readlines()

data_split = [x.strip().split('\t') for x in data[0:]]
df = pd.DataFrame(data_split, columns=['score', 'review'])

In [80]:
df_sampled = df.sample(n=20, random_state=42)

formatted_prompts = [pipeline_prompt.format(
    input=row['review']) for index, row in df_sampled.iterrows()]

In [36]:
template = PromptTemplate(input_variables=["text", "tag"], template="Text:{text}\n{tag}")
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=template,
    suffix="Text: {input}",
    input_variables=["input"],
)

In [37]:
formatted_prompt = prompt.format(input="가게를 운영하는 사장님께서 대신 구매 해달라고 하셔서 구매하게되었는데 가게에 달아놓으니까 이쁘네요")

In [49]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)

In [82]:
responses = []
for prompt in formatted_prompts:
    response = llm(prompt)
    responses.append(response)

# 결과 출력
for response in responses:
    print(response)

--- Logging error ---
Traceback (most recent call last):
  File "/Users/gimsan/Library/Python/3.9/lib/python/site-packages/langchain_core/callbacks/manager.py", line 1997, in _configure
    handler = LangChainTracer(
  File "/Users/gimsan/Library/Python/3.9/lib/python/site-packages/langchain_core/tracers/langchain.py", line 91, in __init__
    self.client = client or get_client()
  File "/Users/gimsan/Library/Python/3.9/lib/python/site-packages/langchain_core/tracers/langchain.py", line 54, in get_client
    _CLIENT = Client()
  File "/Users/gimsan/Library/Python/3.9/lib/python/site-packages/langsmith/client.py", line 534, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "/Users/gimsan/Library/Python/3.9/lib/python/site-packages/langsmith/client.py", line 323, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key must be provided when using hosted LangSmith API

During handling of the above except

 <tag>'부정 리뷰입니다'</tag>
<tag> '긍정 리뷰입니다' </tag>
 <tag>'긍정 리뷰입니다'</tag>
 <tag>'긍정 리뷰입니다'</tag>
 <tag>'긍정 리뷰입니다'</tag>

 <tag>'부정 리뷰입니다'</tag>
 <tag>'부정 리뷰입니다'</tag>
 <tag>'긍정 리뷰입니다'</tag>
 <tag>'긍정 리뷰입니다'</tag>
 <tag> '긍정 리뷰입니다' </tag>
 <tag>'부정 리뷰입니다'</tag>
 <tag>'긍정 리뷰입니다'</tag>
 <tag>'부정 리뷰입니다'</tag>
 <tag>'부정 리뷰입니다'</tag>
 <tag>'부정 리뷰입니다'</tag>
 <tag>'긍정 리뷰입니다'</tag>

<tag>'긍정 리뷰입니다'</tag>


<tag>'긍정 리뷰입니다'</tag>


In [86]:
df_sampled['tag'] = df_sampled['score'].apply(lambda x: '긍정' if int(x) >=3 else '부정')
df_sampled['predict'] = responses
print(df_sampled)

,score,review,tag
119737,2,마감이 안좋아요...실밥도 많고 바느질도 부족한 부분이 몇군데 있네요...교환받기 ...,부정
72272,5,깨끗하게 잘 다듬어져 있어요. 맛도좋고요.,긍정
158154,4,재구매 배송빨라요 길냥이들이 잘먹어요~~ 대용량이라 좋네요~,긍정
65426,5,제품도 빨리 배송해주시고 꼼꼼하게 잘챙겨주셨어요,긍정
30074,5,기타 남 멋지고 예뻐요 여러 사은품도 좋아요,긍정
23677,4,기존것보다 다리를 올려놓으면 푹빠지니깐 무서워서 안올라가요.ㅠㅠ 안고 올려놓으면 가...,긍정
134858,2,상품은잘받았습니다 요청한거와 손잡이방향은다르게왔지만 설치는 잘했습니다. 좀더 신경을...,부정
176418,2,재구매 아기땜에 하루빨리 필요했는데 배송도 오래 걸렸는데 벨크로가 안왔어요 민원처리...,부정
132467,5,좋네요. 사서 방전된 차에 점프 했는데 잘 됐습니다.,긍정
4082,4,저렴하게 잘샀어요ㅎ 쓰던거라 좋아요,긍정
